In [6]:
import tensorflow as tf

# Wildfire Spread Prediction Feature Descriptions

| Feature Key  | Description |
|-------------|-------------|
| `vs`       | **Wind Speed** (m/s) – Measures how fast the wind is moving, affecting fire spread. |
| `th`       | **Theta (Potential Temperature)** – Represents the temperature an air parcel would have if moved adiabatically to a standard pressure level. |
| `population` | **Population Density** – Indicates how many people live in a given area, affecting fire risk and response strategies. |
| `tmmx`     | **Maximum Daily Temperature** (°C) – The highest temperature recorded during the day. |
| `PrevFireMask` | **Previous Fire Presence** (Binary/Mask) – Indicates if there was fire in the area the day before. |
| `elevation` | **Elevation** (meters) – The height above sea level, affecting weather conditions and fire behavior. |
| `sph`      | **Specific Humidity** (kg/kg) – The amount of water vapor per unit of air mass, influencing fuel moisture. |
| `pr`       | **Precipitation** (mm) – The amount of rainfall, which can suppress fire spread. |
| `pdsi`     | **Palmer Drought Severity Index (PDSI)** – A measure of drought conditions, with lower values indicating more severe drought. |
| `erc`      | **Energy Release Component (ERC)** – A fire weather index estimating the potential available energy in live and dead fuels. |
| `FireMask` | **Current Fire Presence** (Binary/Mask) – Indicates if there is an active fire in the area. |
| `NDVI`     | **Normalized Difference Vegetation Index (NDVI)** – A measure of vegetation health, where higher values indicate lush greenery and lower values indicate dry or dead vegetation. |
| `tmmn`     | **Minimum Daily Temperature** (°C) – The lowest temperature recorded during the day. |

---

### Why These Features Matter for Fire Spread Prediction
- **Temperature (`tmmx`, `tmmn`)**: Higher temperatures dry out vegetation, making it more flammable.
- **Humidity (`sph`)**: Lower humidity means drier conditions, increasing fire risk.
- **Wind Speed (`vs`)**: Stronger winds accelerate fire spread and can change fire direction unpredictably.
- **Precipitation (`pr`)**: Rainfall can suppress fire ignition and spread.
- **Drought Index (`pdsi`)**: Long-term dryness can make regions more susceptible to wildfires.
- **Energy Release Component (`erc`)**: Higher ERC values indicate more available fuel energy, leading to more intense fires.
- **Vegetation Health (`NDVI`)**: Green vegetation retains moisture, while dry vegetation is more flammable.
- **Elevation (`elevation`)**: Fires behave differently at higher altitudes due to changes in oxygen levels and terrain.
- **Fire Mask Features (`FireMask`, `PrevFireMask`)**: Indicate whether there were past or present wildfires in a given area, helping models track fire movement.

In [7]:
import tensorflow as tf
import os

# Define dataset directory (not a single file)
tfrecord_dir = "/data/ai_club/fire/next_day_fire_spread_data"

def get_tfrecord_files(prefix):
    """Retrieve sorted TFRecord file paths matching the given prefix."""
    return sorted([
        os.path.join(tfrecord_dir, f) 
        for f in os.listdir(tfrecord_dir) 
        if f.startswith(prefix) and f.endswith(".tfrecord")
    ])

# Automatically get all train, eval, and test files
train_tfrecord_files = get_tfrecord_files("next_day_wildfire_spread_train_")
eval_tfrecord_files = get_tfrecord_files("next_day_wildfire_spread_eval_")
test_tfrecord_files = get_tfrecord_files("next_day_wildfire_spread_test_")

print("Train TFRecord Files:", train_tfrecord_files)
print("Eval TFRecord Files:", eval_tfrecord_files)
print("Test TFRecord Files:", test_tfrecord_files)

# Load first record to check feature structure
def get_feature_names(tfrecord_files):
    """Extract feature names from a TFRecord file."""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_files)
    for raw_record in raw_dataset.take(1):  # Take first record
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        return list(example.features.feature.keys())  # Extract feature names

if train_tfrecord_files:
    feature_names = get_feature_names(train_tfrecord_files[0])
    print("Feature Names:", feature_names)
else:
    print("No training TFRecords found.")

def get_feature_schema(tfrecord_file):
    """Retrieve the feature schema from a TFRecord."""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
    for raw_record in raw_dataset.take(1):  # Inspect first record
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())

        schema = {}
        for key, feature in example.features.feature.items():
            dtype = feature.WhichOneof("kind")
            if dtype == "float_list":
                schema[key] = tf.io.FixedLenFeature([len(feature.float_list.value)], tf.float32)
            elif dtype == "int64_list":
                schema[key] = tf.io.FixedLenFeature([len(feature.int64_list.value)], tf.int64)
            elif dtype == "bytes_list":
                schema[key] = tf.io.FixedLenFeature([], tf.string)
        return schema

if train_tfrecord_files:
    feature_schema = get_feature_schema(train_tfrecord_files[0])
    print("Feature Schema:", feature_schema)
else:
    print("No training TFRecords found.")

def parse_tfrecord_fn(example_proto):
    """Parse a TFRecord using the extracted feature schema."""
    parsed_features = tf.io.parse_single_example(example_proto, feature_schema)

    # Assuming each feature is a 64x64 grid, reshape accordingly
    cnn_input = tf.stack([
        tf.reshape(parsed_features['FireMask'], [64, 64]),
        tf.reshape(parsed_features['NDVI'], [64, 64]),
        tf.reshape(parsed_features['PrevFireMask'], [64, 64])
    ], axis=-1)  # Shape: (64, 64, 3)

    # No LSTM input needed for spatial data
    dense_input = tf.stack([
        parsed_features.get('vs', tf.constant(0.0)), 
        parsed_features.get('tmmx', tf.constant(0.0)), 
        parsed_features.get('tmmn', tf.constant(0.0))
    ], axis=-1)

    # Reshape target to match model output
    target = tf.reduce_mean(parsed_features['FireMask'])  # Example: reduce to a single value

    return {'cnn_input': cnn_input, 'dense_input': dense_input}, target

def load_dataset(filenames, batch_size=32):
    """Load and prepare the TFRecord dataset."""
    if not filenames:
        print("Warning: No files provided for dataset loading.")
        return None
    
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load datasets
train_dataset = load_dataset(train_tfrecord_files)
eval_dataset = load_dataset(eval_tfrecord_files)
test_dataset = load_dataset(test_tfrecord_files)

print("Datasets successfully loaded.")

Train TFRecord Files: ['/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_00.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_01.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_02.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_03.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_04.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_05.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_06.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_07.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_08.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spread_train_09.tfrecord', '/data/ai_club/fire/next_day_fire_spread_data/next_day_wildfire_spr

2025-02-05 18:45:50.229666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13750 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:60:00.0, compute capability: 7.5
2025-02-05 18:45:50.231969: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13750 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:61:00.0, compute capability: 7.5
2025-02-05 18:45:50.234206: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13750 MB memory:  -> device: 2, name: Tesla T4, pci bus id: 0000:da:00.0, compute capability: 7.5
2025-02-05 18:45:50.236306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 13750 MB memory:  -> device: 3, name: Tesla T4, pci bus id: 0000:db:00.0, compute capability: 7.5


Feature Schema: {'vs': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'th': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'population': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'tmmx': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'PrevFireMask': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'elevation': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'sph': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'pr': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'pdsi': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'erc': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'FireMask': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'NDVI': FixedLenFeature(shape=[4096], dtype=tf.float32, default_value=None), 'tmmn': FixedLenFeature(shape=[4096], dtype=

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Model

# Define CNN Model
cnn_input = Input(shape=(64, 64, 3), name="cnn_input")  # Adjust shape based on your data
x = Conv2D(32, (3, 3), activation="relu")(cnn_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation="relu")(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation="relu")(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=cnn_input, outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

# Train the model
model.fit(train_dataset, epochs=20, batch_size=32)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['dense_input'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
2025-02-05 18:45:59.892489: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
2025-02-05 18:46:02.661472: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8074d47b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-05 18:46:02.661541: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-05 18:46:02.661553: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2025-02-05 18:46:02.661561: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (2): Tesla T4, Compute Capability 7.5
2025-02-05 18:46:

    469/Unknown - 20s 13ms/step - loss: 1.0111 - accuracy: 0.0017

2025-02-05 18:46:10.973357: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 1891445747162099651
2025-02-05 18:46:10.973461: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10182094178111851252
2025-02-05 18:46:10.973510: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10600401745972727462
2025-02-05 18:46:10.973522: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 14196073620139872227
2025-02-05 18:46:10.973541: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18079622383691302428


469/469 [==============================] - 20s 14ms/step - loss: 1.0111 - accuracy: 0.0017
Epoch 2/20
469/469 [==============================] - 5s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 3/20
469/469 [==============================] - 5s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 4/20
469/469 [==============================] - 5s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 5/20
469/469 [==============================] - 5s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 6/20
469/469 [==============================] - 4s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 7/20
469/469 [==============================] - 4s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 8/20
469/469 [==============================] - 4s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 9/20
469/469 [==============================] - 5s 8ms/step - loss: 1.0390 - accuracy: 0.0000e+00
Epoch 10/20
206/469 [============>.................] - ETA: 1s - loss: 1.0393

In [ ]:
# Evaluate the model
eval_loss, eval_accuracy = model.evaluate(eval_dataset)
print(f"Evaluation Loss: {eval_loss}, Evaluation Accuracy: {eval_accuracy}")

# Test the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")